# Welcome to the Prognostics Algorithms Package Tutorial

The goal of this notebook is to instruct the user on how to use and extend the NASA Python Prognostics Algorithms Package. 

First some background. The Prognostics Algorithms Package (`prog_algs`) contains tools for performing prognostics (event prediction) using the Prognostics Models Package. `prog_algs` also includes tools for analyzing the performance of prognostics algorithms. 

A few definitions:
* state estimation: The process of estimating the (possibly hidden) state of a system given sensor information on observable states
* prediction: The process of predicting the evolution of a system state with time and the occurance of events. 

The `prog_algs` package has the following primary subpackages
* `prog_algs.state_estimators` - Tools for performing state estimation
* `prog_algs.predictors` - Tools for performing prediction
* `prog_algs.uncertain_data` - Tools for representing data with uncertainty

In addition to the `prog_algs` package, this repo includes examples showing how to use the package (see `examples/`), a template for implementing a new state estimator (`state_estimator_template`), a template for implementing a new predictor (`predictor_template`), documentation (<https://nasa.github.io/prog_algs>), and this tutorial (`tutorial.ipynb`).

Before you start, install `prog_algs` using pip:

      `pip install prog_algs`

or, to use the pre-release, close from GitHub and checkout the dev branch. Then run the following command:
      `pip install -e .`

Now lets get started with some examples

## Prognostics Example 
First, import the prog_algs and the model you intend to use

In [ ]:
from prog_models.models import BatteryCircuit
from prog_algs import *

Next, prepare the model like you did for simulation

In [ ]:
def future_loading(t, x={}):
    # Variable (piece-wise) future loading scheme 
    if (t < 600):
        i = 2
    elif (t < 900):
        i = 1
    elif (t < 1800):
        i = 4
    elif (t < 3000):
        i = 2
    else:
        i = 3
    return {'i': i}

batt = BatteryCircuit()

Now that we have our model ready, we can construct our state estimator:

In [ ]:
filt = state_estimators.UnscentedKalmanFilter(batt, batt.parameters['x0'])

The filter estimate function can then be run when there is updated data. Each iteration it will produce a new estimate of the system state (with uncertainty). For example:

In [ ]:
import matplotlib.pyplot as plt  # For plotting

print("Prior State:", filt.x.mean)
print('\tSOC: ', batt.event_state(filt.x.mean)['EOD'])
fig = filt.x.plot_scatter(label='prior')
example_measurements = {'t': 32.2, 'v': 3.915}
t = 0.1
filt.estimate(t, future_loading(t), example_measurements)
print("Posterior State:", filt.x.mean)
print('\tSOC: ', batt.event_state(filt.x.mean)['EOD'])
filt.x.plot_scatter(fig= fig, label='posterior')

fig.show()

Typically in a prognostics application the state estimation step is run whenever there is new data.

That's the state estimation step- now lets prepare for prediction. 

In [ ]:
# Create predictor 
mc = predictors.MonteCarlo(batt)

Now lets use the constructed mc predictor to perform a single prediction. Here we're setting dt to 0.25. Note this may take up to a minute

In [ ]:
(times, inputs, states, outputs, event_states, toe) = mc.predict(filt.x, future_loading, dt=0.05, n_samples=20)

The predict function returns predictions of future inputs, states, outputs, and event_states at each save point. For sample-based predictors like the monte carlo, these can be accessed like an array with the format `[sample #][time]` so that `states[m][n]` corresponds to the state for sample `m` at time `times[m][n]`. Alternately, use the method `snapshot` to get a  single point in time. e.g., 

    `state = states.snapshot(3)`

In this case the state snapshot corresponds to time `times[3]`. The snapshot method returns type UncertainData. 

The `predict` method also returns Time of Event (ToE) as a type UncertainData, representing the predicted time of event (for each event predicted), with uncertainty.

Next, let's use the metrics package to analyse the ToE

In [ ]:
print("\nEOD Predictions (s):")
print('\tPortion between 3005.2 and 3005.6: ', toe.percentage_in_bounds([3005.2, 3005.6]))
print('\tAssuming ground truth 3005.25: ', toe.metrics(ground_truth = 3005.25))
from prog_algs.metrics import prob_success 
print('\tP(Success) if mission ends at 3005.25: ', prob_success(toe, 3005.25))

These analysis methods applied to ToE can also be applied to anything of type UncertainData (e.g., state snapshot). 

You can also visualize the results in a variety of different ways. For example, state transition

In [ ]:
fig = states.snapshot(0).plot_scatter(label = "t={:.0f}".format(int(times[0])))
for i in range(1, 4):
    index = int(len(times)/4*i)
    states.snapshot(index).plot_scatter(fig=fig, label = "t={:.0f}".format(times[index]))
states.snapshot(-1).plot_scatter(fig = fig, label = "t={:.0f}".format(int(times[-1])))
fig.show()

Or time of event

In [ ]:
fig = toe.plot_hist()
fig.show()

Note, for this event, there is only one event (End of Discharge). Many models have multiple events that can be predicted. For these models, ToE for each event is returned and can be analyzed. 

Alternately, a specific event (or events) can be specified for prediction. See `examples.predict_specific_event` for more details.

## Extending - Adding a new state estimator
New state estimators can be created by extending the state_estimator interface. As an example lets use a really dumb state estimator that adds random noise each step - and accepts the state that is closest. 

First thing we need to do is import the StateEstimator parent class

In [ ]:
from prog_algs.state_estimators.state_estimator import StateEstimator

Next we select how state will be represented. In this case there's no uncertainty- it's just one state, so we represent it as a scaler. Import the appropriate class

In [ ]:
from prog_algs.uncertain_data import ScalarData

Now we construct the class, implementing the functions of the state estimator template (`state_estimator_template.py`)

In [ ]:
import random 

class BlindlyStumbleEstimator(StateEstimator):
    def __init__(self, model, x0):
        self.m = model
        self.state = x0

    def estimate(self, t, u, z):
        # Generate new candidate state
        x2 = {key : float(value) + 10*(random.random()-0.5) for (key,value) in self.state.items()}

        # Calculate outputs
        z_est = self.m.output(self.state)
        z_est2 = self.m.output(x2)

        # Now score them each by how close they are to the measured z
        z_est_score = sum([abs(z_est[key] - z[key]) for key in self.m.outputs])
        z_est2_score = sum([abs(z_est2[key] - z[key]) for key in self.m.outputs])

        # Now choose the closer one
        if z_est2_score < z_est_score: 
            self.state = x2

    @property
    def x(self):
        return ScalarData(self.state)


Great, now let's try it out using the model from earlier. with an initial state of all 0s. It should slowly converge towards the correct state

In [ ]:
x0 = {key: 0 for key in batt.states}
se = BlindlyStumbleEstimator(batt, x0)

for i in range(25):
    u = {'i': 0}
    z = {'t': 18.95, 'v': 4.183}
    se.estimate(i, u, z)
    print(se.x.mean)
    print("\tcorrect: {'tb': 18.95, 'qb': 7856.3254, 'qcp': 0, 'qcs': 0}")

Like the example above with StateEstimators, Predictors can be extended by subclassing the Predictor class. Copy `predictor_template.py` as a starting point.

## Conclusion
This is just the basics, there's much more to learn. Please see the documentation at <https://nasa.github.io/prog_algs> and the examples in the `examples/` folder for more details on how to use the package, including:
* `examples.basic_example` : A basic Example using prog_algs for Prognsotics 
* `examples.benchmarking_example` : An example benchmarking the performance of prognsotis algorihtms
* `examples.eol_event` : An example where a model has multiple events, but the user is only interested in predicting the time when the first event occurs (whatever it is).
* `examples.measurement_eqn_example` : An example where not every output is measured or measurements are not in the same format as outputs, so a measurement equation is defined to translate between outputs and what's measured. 
* `examples.new_state_estimator_example` : An example of extending StateEstimator to create a new state estimator class
* `examples.playback` : A full example performing prognostics using playback data.
* `examples.predict_specific_event` : An example where the model has multiple events, but the user is only interested in predicting a specific event (or events).
* `examples.thrown_object_example` : An example performing prognostics with the simplified ThrownObject model
* `examples.utpredictor` : An example using the Unscented Transform Predictor for prediction.

Thank you for trying out this tutorial. Open an issue on github (<https://github.com/nasa/prog_algs/issues>) or email Chris Teubert (christopher.a.teubert@nasa.gov) with any questions or issues.